In [6]:
from LWP import LWP

import joblib
import numpy as np
import pandas as pd

In [ ]:
def keep(x, pred, upper_bound=0.99999):
    class_pred = np.argmax(pred, axis=1)
    keep_x = []
    keep_y = []
    for i, c in enumerate(class_pred):
        if pred[i][c] > upper_bound:
            keep_x.append(x[i])
            keep_y.append(c)

    return np.array(keep_x), np.array(keep_y)

#

Next, starting with f10 from task 1 as the initial model, repeat the same procedure using the datasets D11;D12; : : : ;D20 (again, one dataset at a time) to learn models f11; f12; : : : ; f20.

Just like you did for task 1, once you have learned f11; f12; : : : ; f20, apply each model fi that belongs to {ff11; f12; : : : ; f20} on the ith heldout dataset ^Di as well as the previous heldout datasets ^Dj ;j<i(e.g., apply f13 on ^D13 and also on ^D1; ^D2; : : : ; ^D12), and report the accuracies of each of these models on each of these heldout datasets. In your report, show the various accuracies in form of a matrix (with 10 rows representing the 10 models and 20 columns representing the 20 held-out datasets).

In [26]:
# Load f10 as the initial model for Task 2
f10 = joblib.load("f10_model.pkl")
current_model = f10

# Placeholder for accuracy matrix (10 models x 20 datasets)
accuracy_matrix_task2 = np.zeros((10, 20))

for i in range(11, 21):  # Loop through D11 to D20
    print(f"Processing dataset D{i}")
    
    # Load the training dataset (D11 to D20 correspond to files 1_train_data.npz to 10_train_data.npz in part_two_dataset)
    train_data = np.load(f"Augmented_Data/part_two_dataset/train_data/{i-10}_train_data.npz")
    train_x = train_data['data']
    
    # Load the evaluation dataset (D̂11 to D̂20 correspond to files 1_eval_data.npz to 10_eval_data.npz in part_two_dataset)
    eval_data = np.load(f"Augmented_Data/part_two_dataset/eval_data/{i-10}_eval_data.npz")
    eval_x, eval_y = eval_data['data'], eval_data['targets']
    
    # Predict labels for the current dataset
    preds_prob = current_model.predict(train_x)
    keep_x, keep_y = keep(train_x, preds_prob, upper_bound=0.99999)
    
    # Train the next model
    next_model = current_model.copy()
    next_model.fit(keep_x, keep_y)
    
    # Evaluate the updated model on held-out datasets D̂1 to D̂i
    for j in range(1, i + 1):  # Evaluate on D̂1 to D̂i
        if j <= 10:
            eval_data_j = np.load(f"Augmented_Data/part_one_dataset/eval_data/{j}_eval_data.npz")
        else:
            eval_data_j = np.load(f"Augmented_Data/part_two_dataset/eval_data/{j-10}_eval_data.npz")
        eval_x_j, eval_y_j = eval_data_j['data'], eval_data_j['targets']
        accuracy = next_model.evaluate(eval_x_j, eval_y_j)
        accuracy_matrix_task2[i - 11, j - 1] = accuracy
        print(f"Accuracy of f{i} on D̂{j}: {accuracy}")
    
    # Update the current model
    current_model = next_model

# Save the accuracy matrix for Task 2
np.save("accuracy_matrix_task2.npy", accuracy_matrix_task2)

# Display the accuracy matrix for Task 2
print("Final Accuracy Matrix for Task 2:")
print(accuracy_matrix_task2)


Processing dataset D11
Accuracy of f11 on D̂1: 0.8556
Accuracy of f11 on D̂2: 0.8744
Accuracy of f11 on D̂3: 0.8672
Accuracy of f11 on D̂4: 0.868
Accuracy of f11 on D̂5: 0.8692
Accuracy of f11 on D̂6: 0.858
Accuracy of f11 on D̂7: 0.8568
Accuracy of f11 on D̂8: 0.8676
Accuracy of f11 on D̂9: 0.8684
Accuracy of f11 on D̂10: 0.874
Accuracy of f11 on D̂11: 0.7332
Processing dataset D12
Accuracy of f12 on D̂1: 0.8528
Accuracy of f12 on D̂2: 0.8744
Accuracy of f12 on D̂3: 0.8668
Accuracy of f12 on D̂4: 0.8672
Accuracy of f12 on D̂5: 0.8684
Accuracy of f12 on D̂6: 0.856
Accuracy of f12 on D̂7: 0.858
Accuracy of f12 on D̂8: 0.866
Accuracy of f12 on D̂9: 0.8672
Accuracy of f12 on D̂10: 0.8728
Accuracy of f12 on D̂11: 0.7332
Accuracy of f12 on D̂12: 0.512
Processing dataset D13
Accuracy of f13 on D̂1: 0.8528
Accuracy of f13 on D̂2: 0.8732
Accuracy of f13 on D̂3: 0.866
Accuracy of f13 on D̂4: 0.866
Accuracy of f13 on D̂5: 0.8684
Accuracy of f13 on D̂6: 0.8552
Accuracy of f13 on D̂7: 0.8576
Accur

In [27]:
# Create a DataFrame for a nicer format
accuracy_matrix_df = pd.DataFrame(
    accuracy_matrix_task2,
    columns=[f"D̂{j+1}" for j in range(accuracy_matrix_task2.shape[1])],  # Held-out datasets
    index=[f"f{i+11}" for i in range(accuracy_matrix_task2.shape[0])]  # Models from f11 to f20
)
# Replace 0.0000 with an empty string for better visualization
accuracy_matrix_df = accuracy_matrix_df.replace(0.0000, "")


In [28]:
accuracy_matrix_df

,D̂1,D̂2,D̂3,D̂4,D̂5,D̂6,D̂7,D̂8,D̂9,D̂10,D̂11,D̂12,D̂13,D̂14,D̂15,D̂16,D̂17,D̂18,D̂19,D̂20
f11,0.8556,0.8744,0.8672,0.8680,0.8692,0.8580,0.8568,0.8676,0.8684,0.8740,0.7332,,,,,,,,,
f12,0.8528,0.8744,0.8668,0.8672,0.8684,0.8560,0.8580,0.8660,0.8672,0.8728,0.7332,0.512,,,,,,,,
f13,0.8528,0.8732,0.8660,0.8660,0.8684,0.8552,0.8576,0.8664,0.8660,0.8724,0.7312,0.5112,0.7784,,,,,,,
f14,0.8512,0.8728,0.8656,0.8648,0.8688,0.8560,0.8560,0.8656,0.8656,0.8720,0.7300,0.5104,0.7788,0.8232,,,,,,
f15,0.8516,0.8720,0.8660,0.8656,0.8668,0.8552,0.8540,0.8652,0.8636,0.8704,0.7312,0.5116,0.7772,0.8228,0.8504,,,,,
f16,0.8492,0.8732,0.8664,0.8644,0.8664,0.8552,0.8532,0.8640,0.8636,0.8696,0.7344,0.5128,0.7756,0.8216,0.8508,0.7236,,,,
f17,0.8500,0.8728,0.8652,0.8640,0.8668,0.8552,0.8532,0.8640,0.8632,0.8688,0.7336,0.5112,0.7756,0.8216,0.8508,0.7248,0.7736,,,
f18,0.8492,0.8720,0.8636,0.8628,0.8660,0.8552,0.8528,0.8632,0.8628,0.8688,0.7320,0.51,0.774,0.8224,0.8492,0.7232,0.7724,0.7416,,
f19,0.8484,0.8724,0.8648,0.8620,0.8664,0.8536,0.8536,0.8620,0.8612,0.8680,0.7308,0.5104,0.774,0.8212,0.8484,0.7224,0.7724,0.7388,0.6496,
f20,0.8492,0.8708,0.8640,0.8628,0.8648,0.8524,0.8540,0.8608,0.8624,0.8672,0.7320,0.5116,0.7736,0.8216,0.8464,0.7224,0.7716,0.7384,0.6468,0.8284
